In [19]:
#AUTOR:Omar Rodrigo Sorchini Puente

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:


import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [21]:

import glob
folder_path_plg = "/content/drive/MyDrive/DATASET_IA/documentos-con texto de otros/"
# Obtener los nombres de los archivos en la carpeta
file_names_plg = glob.glob(folder_path_plg + "/*")

folder_path_gen = "/content/drive/MyDrive/DATASET_IA/documentos-genuinos/"
# Obtener los nombres de los archivos en la carpeta
file_names_gen = glob.glob(folder_path_gen + "/*")

In [22]:

#Dataset from originals
original_texts_nf = []
for file_path in file_names_gen:
  with open(file_path, "r") as file:
      content = file.read()
      original_texts_nf.append(content)



In [23]:

# Imprimir los nombres de los archivos
plagiarized_texts_nf = []
for file_name in file_names_plg:
  with open(file_name, "r") as file:
      content = file.read()
      plagiarized_texts_nf.append(content)

print(len(plagiarized_texts_nf))



99


In [28]:
#AUTOR:Eric Alexis Castañeda Bravo 

import re
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def lematize(text):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) for word in text.split()]


def limpiarText(texto_sucio):
  text = re.sub(r"[^\w\s]", "", texto_sucio)
  text = re.sub(r"\d+", "", text)
  text = text.replace("\n", " ")
  textoLimpio = text.lower()

  #textoLimpio = re.findall(r'\w+\W*|\w+', texto_sucio)
  return textoLimpio

original_texts = []

for o in original_texts_nf:
  #print("aaaaaaaaaaaaaaaaaaaaaaaaa")
  #print(lematize(o))
  original_texts.append(lematize(limpiarText(o)))

plagiarized_texts = []
for o in plagiarized_texts_nf:
  plagiarized_texts.append(lematize(limpiarText(o)))


print(original_texts)
#print(len(plagiarized_texts))

[nltk_data] Downloading package wordnet to /root/nltk_data...


[['the', 'purpose', 'of', 'this', 'paper', 'is', 'to', 'analyse', 'the', 'stateoftheart', 'technique', 'used', 'to', 'detect', 'plagiarism', 'in', 'term', 'of', 'their', 'limitation', 'feature', 'taxonomy', 'and', 'process', 'the', 'method', 'used', 'to', 'execute', 'this', 'study', 'consisted', 'of', 'a', 'comprehensive', 'search', 'for', 'relevant', 'literature', 'via', 'six', 'online', 'database', 'repository', 'namely', 'ieee', 'xplore', 'acm', 'digital', 'library', 'sciencedirect', 'ei', 'compendex', 'web', 'of', 'science', 'and', 'springer', 'using', 'search', 'string', 'obtained', 'from', 'the', 'subject', 'of', 'discussion', 'the', 'finding', 'revealed', 'that', 'existing', 'plagiarism', 'detection', 'technique', 'require', 'further', 'enhancement', 'a', 'existing', 'technique', 'are', 'incapable', 'of', 'efficiently', 'detecting', 'plagiarised', 'idea', 'figure', 'table', 'formula', 'and', 'scanned', 'document', 'the', 'contribution', 'of', 'this', 'study', 'lie', 'in', 'it', 

In [29]:

# Combine original and plagiarized texts
texts = original_texts + plagiarized_texts

# Create labels (0 for original, 1 for plagiarized)
labels = np.array([0] * len(original_texts) + [1] * len(plagiarized_texts))

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_sequence_length = max(len(sequence) for sequence in sequences)
sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Split data into train and test sets
train_size = int(0.8 * len(texts))
train_sequences = sequences[:train_size]
train_labels = labels[:train_size]
test_sequences = sequences[train_size:]
test_labels = labels[train_size:]


In [30]:

# Build the model
model = keras.Sequential([
    keras.layers.Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_sequence_length),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [32]:
#AUTOR:Omar Rodrigo Sorchini Puente

# Train the model
model.fit(train_sequences, train_labels, epochs=100, batch_size=16, validation_data=(test_sequences, test_labels),
          callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Epoch 1/100
11/11 [==============================] - 0s 20ms/step - loss: 0.5794 - accuracy: 0.6857 - val_loss: 1.0949 - val_accuracy: 0.0000e+00
Epoch 2/100
11/11 [==============================] - 0s 18ms/step - loss: 0.5660 - accuracy: 0.6857 - val_loss: 1.0443 - val_accuracy: 0.0000e+00
Epoch 3/100
11/11 [==============================] - 0s 15ms/step - loss: 0.5506 - accuracy: 0.6857 - val_loss: 0.9870 - val_accuracy: 0.0000e+00
Epoch 4/100
11/11 [==============================] - 0s 16ms/step - loss: 0.5294 - accuracy: 0.6857 - val_loss: 0.9845 - val_accuracy: 0.0000e+00
Epoch 5/100
11/11 [==============================] - 0s 16ms/step - loss: 0.5056 - accuracy: 0.6857 - val_loss: 0.9300 - val_accuracy: 0.0000e+00
Epoch 6/100
11/11 [==============================] - 0s 15ms/step - loss: 0.4767 - accuracy: 0.6857 - val_loss: 0.8830 - val_accuracy: 0.0000e+00
Epoch 7/100
11/11 [==============================] - 0s 16ms/step - loss: 0.4456 - accuracy: 0.6914 - val_loss: 0.8086 - val

In [33]:
# Evaluate the model
_, accuracy = model.evaluate(test_sequences, test_labels)
print('Accuracy:', accuracy)



2/2 [==============================] - 0s 8ms/step - loss: 0.0051 - accuracy: 1.0000
Accuracy: 1.0


In [34]:
#AUTOR:Eric Alexis Castañeda Bravo 

# Function to predict plagiarism
def detect_plagiarism(text):
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    prediction = model.predict(sequence)[0][0]
    print(prediction)
    if prediction < 0.5:
        return "Original"
    else:
        return "Plagiarized"

# Example usage
input_text = "How to measure similarity between two sequences is a key question in information theory and in computer science. Based on Kolmogorov complexity, we have proposed a metric to answer this question and prove it to be universal. A substantial tool is the documents similarity metric applied in fields such as plagiarism detection, problems necessary to capture the semantic, in relation to documents, determining the topic, or structural, or syntactic likeness of texts. To evaluate the proposed method, we construct a comprehensive dataset comprising documents from diverse domains, including academic research papers, news articles, and online forums. We simulate various plagiarism scenarios, such as paraphrasing, sentence reordering, and word substitution, to assess the robustness and effectiveness of our approach. Of different linguistic patterns in plagiarism committing, the taxonomy deep understanding supports, an example is example, texts changing into semantically equivalent but with different organization, texts shortening with concept specification and generalization, and ideas adopting, and of others, key contributions."
input_text2 = "There are many variations of passages of Lorem Ipsum available, but the majority have suffered alteration in some form, by injected humour, or randomised words which don't look even slightly believable. If you are going to use a passage of Lorem Ipsum, you need to be sure there isn't anything embarrassing hidden in the middle of text. All the Lorem Ipsum generators on the Internet tend to repeat predefined chunks as necessary, making this the first true generator on the Internet. It uses a dictionary of over 200 Latin words, combined with a handful of model sentence structures, to generate Lorem Ipsum which looks reasonable. The generated Lorem Ipsum is therefore always free from repetition, injected humour, or non-characteristic words etc."
input_text3 = "Plagiarism of digital documents seems a serious problem in today's era. Plagiarism refers to the use of someone's data, language and writing without proper acknowledgment of the original source. Plagiarism of another author's original work is one of the biggest problems in publishing, science, and education. Plagiarism can be of different types. This paper presents a different approach for measuring semantic similarity between words and their meanings. Existing systems are based on the traditional approach. This approach offers the opportunity to tie ethical issues to technical content at the moment students learn it, and to have students engage with these issues repeatedly throughout their degree. However, little is known about the effect of embedded ethics education on students. Therefore, two algorithms are presented: BERT (Bidirectional Encoder Representations from Transformers) which uses a neural network to process the natural language of the texts to capture the context of the words and understand the relationships between them as well as the algorithm USE (Universal Sentence Encoder) which converts sentences into vector representations to perform a semantic analysis."
input_text4 = "Our results suggest that the mean similarity index among postgraduate theses in Austria is 8.78 (Standard deviation 4.91), while the mean similarity index among graduate theses prepared in Turkey is 25.10 ( Standard deviation 9.85). In fact, our analysis indicates that 91% (n 113) of theses prepared at Austrian universities and only 13% (n 17) of theses prepared at Turkish universities failed the acceptable similarity rate of 15%. The fact that 87% of the theses written in Turkey are dramatically similar to the available resources shows that many of the studies carry potential risks in terms of originality and plagiarism. In this article, we discuss the architecture of the APAS web application with a focus on scalability issues. We review foundational features like dynamic analysis and unreliable code execution, as well as more complex cases like static analysis and plagiarism detection, and summarize the lessons learned from the previous six years of research. We identify scalability challenges, show how they have been addressed in APAS Edgar, and then propose general architectural solutions, building blocks, and patterns to address those challenges."
input_text5 = "It describes the experiment of building a software capable of generating leads and newspaper titles in an automated fashion from information obtained from the Internet. The theoretical possibility Lage already provided by the end of last century is based on relatively rigid and simple structure of this type of story construction, which facilitates the representation or translation of its syntax in terms of instructions that the computer can execute. The paper also discusses the relationship between society, technique and technology, making a brief history of the introduction of digital solutions in newsrooms and their impacts. The development was done with the Python programming language and NLTK- Natural Language Toolkit library - and used the results of the Brazilian Soccer Championship 2013 published on an internet portal as a data source."
input_text6 = "Plagiarism of digital documents seems a serious problem in today's era. Plagiarism refers to the use of someone's data, language and writing without proper acknowledgment of the original source. Plagiarism of another author's original work is one of the biggest problems in publishing, science, and education. Plagiarism can be of different types. This paper presents a different approach for measuring semantic similarity between words and their meanings. Existing systems are based on the traditional approach. This approach offers the opportunity to tie ethical issues to technical content at the moment students learn it, and to have students engage with these issues repeatedly throughout their degree. However, little is known about the effect of embedded ethics education on students. Therefore, two algorithms are presented: BERT (Bidirectional Encoder Representations from Transformers) which uses a neural network to process the natural language of the texts to capture the context of the words and understand the relationships between them as well as the algorithm USE (Universal Sentence Encoder) which converts sentences into vector representations to perform a semantic analysis."
input_text7 = "Short text similarity plays an important role in natural language processing (NLP). It has been applied in many fields. Due to the lack of sufficient context in the short text, it is difficult to measure the similarity. The use of semantics similarity to calculate textual similarity has attracted the attention of academia and industry and achieved better results. In this survey, we have conducted a comprehensive and systematic analysis of semantic similarity. We first propose three categories of semantic similarity: corpus-based, knowledge-based, and deep learning (DL)-based. We analyze the pros and cons of representative and novel algorithms in each category. Our analysis also includes the applications of these similarity measurement methods in other areas of NLP. We then evaluate state-of-the-art DL methods on four common datasets, which proved that DL-based can better solve the challenges of the short text similarity, such as sparsity and complexity. Especially, bidirectional encoder representations from transformer model can fully employ scarce information of short texts and semantic information and obtain higher accuracy and F1 value. We finally put forward some future directions."

result = detect_plagiarism(input_text)
print("Result:", result)

result2 = detect_plagiarism(input_text2)
print("Result:", result2)

result3 = detect_plagiarism(input_text3)
print("Result:", result3)

result4 = detect_plagiarism(input_text4)
print("Result:", result4)

result5 = detect_plagiarism(input_text5)
print("Result:", result5)

result6 = detect_plagiarism(input_text6)
print("Result:", result6)

result7 = detect_plagiarism(input_text7)
print("Result:", result7)

1/1 [==============================] - 0s 92ms/step
0.9980587
Result: Plagiarized
1/1 [==============================] - 0s 24ms/step
0.08179497
Result: Original
1/1 [==============================] - 0s 24ms/step
0.994515
Result: Plagiarized
1/1 [==============================] - 0s 25ms/step
0.99621785
Result: Plagiarized
1/1 [==============================] - 0s 26ms/step
3.6016387e-05
Result: Original
1/1 [==============================] - 0s 23ms/step
0.994515
Result: Plagiarized
1/1 [==============================] - 0s 23ms/step
0.0029566193
Result: Original


In [35]:
#AUTOR:Daniel García Barajas

folder_path_gen = "/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos"
# Obtener los nombres de los archivos en la carpeta
file_names_gen = glob.glob(folder_path_gen + "/*")


#Dataset from originals
pruebas = []
for file_path in file_names_gen:
  print(file_path)
  with open(file_path, "r") as file:
      content = file.read()
      pruebas.append(content)

for t in pruebas:
  result = detect_plagiarism(limpiarText(t))
  print("Result:", result)

print("SIN TEXTO LIMPIO")

for t in pruebas:
  result = detect_plagiarism(t)
  print("Result:", result)

/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-12.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-04.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-15.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-14.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-08.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-07.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-01.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-11.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-06.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-05.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-03.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-13.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-09.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-10.txt
/content/drive/MyDrive/DATASET_IA/docmentos-sospechosos/FID-02